https://bit.ly/RSML-3-colab

# Коллаборативная фильтрация

## Модель item-to-item

In [3]:
#!poetry add tqdm

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [3]:
#!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

In [4]:
#!unzip MovieLens.zip

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [6]:
movies_with_ratings.title.value_counts()

title
Forrest Gump (1994)                      329
Shawshank Redemption, The (1994)         317
Pulp Fiction (1994)                      307
Silence of the Lambs, The (1991)         279
Matrix, The (1999)                       278
                                        ... 
We're Back! A Dinosaur's Story (1993)      1
American Hardcore (2006)                   1
Shanghai Surprise (1986)                   1
Let's Get Harry (1986)                     1
Andrew Dice Clay: Dice Rules (1991)        1
Name: count, Length: 9719, dtype: int64

In [7]:
movies_with_ratings.userId.value_counts()

userId
414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
53       20
207      20
431      20
442      20
189      20
Name: count, Length: 610, dtype: int64

In [8]:
num_users = movies_with_ratings.userId.unique().shape[0]
num_users

610

In [9]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [10]:
movie_vector = {}

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)

    for i in range(len(group.userId.values)):
        u = group.userId.values[i]
        r = group.rating.values[i]
        movie_vector[movie][int(u - 1)] = r

100%|█████████████████████████████████████| 9719/9719 [00:05<00:00, 1820.95it/s]


In [11]:
movie_vector['Toy Story (1995)']

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ,
       3. , 0. , 0. , 0. , 5. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. ,
       5. , 0. , 0. , 5. , 3. , 4. , 5. , 0. , 0. , 0. , 3. , 0. , 0. ,
       0. , 3. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 0. ,
       4. , 0. , 2.5, 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0.5, 0. , 4. ,
       0. , 0. , 0. , 2.5, 0. , 0. , 0. , 4. , 0. , 0. , 3. , 3. , 4. ,
       0. , 3. , 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0. , 0. , 4. , 0. ,
       0. , 0. , 4. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 3. ,
       0. , 2. , 0. , 3. , 4. , 0. , 4. , 0. , 0. , 3. , 4. , 0. , 0. ,
       3.5, 5. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 2. , 0. , 3. , 4. ,
       0. , 0. , 4.5, 4. , 4. , 0. , 0. , 0. , 0. , 5. , 3.5, 0. , 4.5,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 4. , 0. , 0.

In [12]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [13]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    if key == my_fav_film:
        continue

    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

len(distances)

100%|████████████████████████████████████| 9719/9719 [00:00<00:00, 33052.15it/s]


9718

In [14]:
distances[:10]

[0.9223029947150826,
 0.9300726952435743,
 0.950553928616633,
 1.0,
 1.0,
 0.968921197886033,
 0.877349727371666,
 0.9378423957720661,
 0.7578680102442159,
 0.9065861273541315]

In [15]:
best_indexes = np.argsort(distances)[:10]
best_indexes

array([5511, 5571,  420, 4736, 5203,  405, 6864, 5204, 4322, 4737])

In [16]:
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

('Matrix, The (1999)', 0.28606257328028306)
('Memento (2000)', 0.33040704494686746)
('American History X (1998)', 0.3509458374819996)
('Kill Bill: Vol. 1 (2003)', 0.3602624134566229)
('Lord of the Rings: The Fellowship of the Ring, The (2001)', 0.3642558063841451)
('American Beauty (1999)', 0.3744505150386159)
('Pulp Fiction (1994)', 0.37678014788325587)
('Lord of the Rings: The Return of the King, The (2003)', 0.37798413607448667)
('Inception (2010)', 0.3845825728533704)
('Kill Bill: Vol. 2 (2004)', 0.38565615909903594)


## Библиотка Surprise

https://surpriselib.com/

- Алгоритмы https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [18]:
#!pip install surprise

In [7]:
#!poetry add numpy

In [17]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [18]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [19]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [20]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


### Подготовка объекта для работы surprise

В датасете должны остаться только три колонки: пользователи, фильмы и рейтинг

>В surprise's Dataset не получится загрузить что-либо, состоящее не из трех колонок

In [21]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [22]:
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


Reader будет выполнять нормализацию данных. Поэтому нам нужны максимальное и минимальное значения рейтинга.

In [23]:
ratings.rating.min()

0.5

In [24]:
ratings.rating.max()

5.0

Создаем объект Dataset

In [25]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [27]:
type(data)

surprise.dataset.DatasetAutoFolds

Делим Dataset на обечение и на тест

In [28]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [29]:
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

### Обучение user_based модели

In [30]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [31]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8985


0.898482528669153

In [33]:
test_pred[:3]

[Prediction(uid=542, iid='Kill Bill: Vol. 1 (2003)', r_ui=2.5, est=3.7592260572900007, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid=469, iid='Bridge on the River Kwai, The (1957)', r_ui=4.0, est=4.168241951992395, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=317, iid='Scarface (1983)', r_ui=5.0, est=4.062599058241541, details={'actual_k': 49, 'was_impossible': False})]

In [34]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.474085079084162, details={'actual_k': 50, 'was_impossible': False})

### Обучение item_based модели

In [35]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [36]:
test_pred = algo.test(testset)

In [37]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8982


0.898181158065249

In [38]:
new_pred = algo.predict(uid=2, iid='Fight Club (1999)')
new_pred

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.2619917274785735, details={'actual_k': 24, 'was_impossible': False})

In [39]:
accuracy.rmse(new_pred, verbose=True)

TypeError: cannot unpack non-iterable int object

In [40]:
new_pred = algo.predict(uid=20000, iid='Fight Club (1999)')
new_pred

Prediction(uid=20000, iid='Fight Club (1999)', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [41]:
new_pred = algo.predict(uid=2, iid='unkown')
new_pred

Prediction(uid=2, iid='unkown', r_ui=None, est=3.498958694897605, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [42]:
trainset.global_mean

3.498958694897605

#### Рекомендации

In [43]:
def generate_recommendation(uid, model, dataset, thresh=4.5, amount=5):
    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    np.random.shuffle(titles)

    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title)
        rating = review_prediction.est

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))

            if len(rec_list) >= amount:
                return rec_list

In [46]:
generate_recommendation(2, algo, dataset, thresh=4.8)

[("Jonah Who Will Be 25 in the Year 2000 (Jonas qui aura 25 ans en l'an 2000) (1976)",
  5.0),
 ('Song of the Little Road (Pather Panchali) (1955)', 5.0),
 ('Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986)',
  5.0),
 ('King of Hearts (1966)', 5.0),
 ('Holy Motors (2012)', 5.0)]

Дополнительные ссылки:
1. https://sales-generator.ru/blog/rekomendatelnye-sistemy/
2. https://habr.com/ru/companies/prequel/articles/567648/
3. https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101
4. https://habr.com/ru/companies/prequel/articles/573880/